In [1]:
!pip install torchvision scikit-learn


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import zipfile
zip_path = "/content/drive/MyDrive/Task_B.zip"  # Adjust if in subfolder
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content")


In [4]:
import torch.nn as nn
import torchvision.models as models

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        base_model = models.resnet18(pretrained=True)
        base_model.fc = nn.Identity()  # remove final FC
        self.encoder = base_model
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x1, x2):
        f1 = self.encoder(x1)
        f2 = self.encoder(x2)
        dist = torch.abs(f1 - f2)
        return self.classifier(dist)


In [5]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import random

class FaceConSiameseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.person_folders = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]
        self.pairs = self.create_pairs()

    def create_pairs(self):
        pairs = []
        for person_dir in self.person_folders:
            frontal_img = None
            for file in os.listdir(person_dir):
                if file.lower().endswith(('.jpg', '.png')) and "frontal" in file.lower():
                    frontal_img = os.path.join(person_dir, file)
                    break

            distortion_dir = os.path.join(person_dir, "distortion")
            if not frontal_img or not os.path.exists(distortion_dir):
                continue

            distorted_images = [
                os.path.join(distortion_dir, f)
                for f in os.listdir(distortion_dir)
                if f.lower().endswith(('.jpg', '.png'))
            ]

            for dimg in distorted_images:
                pairs.append((frontal_img, dimg, 1))

            neg_person = random.choice([p for p in self.person_folders if p != person_dir])
            neg_dist_dir = os.path.join(neg_person, "distortion")
            if os.path.exists(neg_dist_dir):
                neg_imgs = [os.path.join(neg_dist_dir, f) for f in os.listdir(neg_dist_dir) if f.lower().endswith(('.jpg', '.png'))]
                if neg_imgs:
                    neg_img = random.choice(neg_imgs)
                    pairs.append((frontal_img, neg_img, 0))

        print(f"✅ Created {len(pairs)} pairs")
        return pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        path1, path2, label = self.pairs[idx]
        img1 = Image.open(path1).convert("RGB")
        img2 = Image.open(path2).convert("RGB")

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2, torch.tensor(label, dtype=torch.float32)


In [6]:
import torch.nn as nn
import torchvision.models as models

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        resnet = models.resnet18(weights='IMAGENET1K_V1')
        resnet.fc = nn.Identity()  # remove final FC
        self.backbone = resnet
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, x1, x2):
        feat1 = self.backbone(x1)
        feat2 = self.backbone(x2)
        diff = torch.abs(feat1 - feat2)
        out = self.fc(diff)
        return out


In [7]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = FaceConSiameseDataset("/content/Task_B/train", transform=transform)
val_dataset = FaceConSiameseDataset("/content/Task_B/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def evaluate(model, loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x1, x2, labels in loader:
            x1, x2, labels = x1.to(device), x2.to(device), labels.to(device)
            outputs = model(x1, x2).squeeze()
            preds = torch.sigmoid(outputs) > 0.5
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "F1": f1_score(y_true, y_pred)
    }

# 🔁 Training Loop
for epoch in range(5):
    model.train()
    total_loss = 0
    for x1, x2, labels in train_loader:
        x1, x2, labels = x1.to(device), x2.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(x1, x2).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")
    metrics = evaluate(model, val_loader)
    print("Validation:", metrics)


✅ Created 713 pairs
✅ Created 224 pairs
Epoch 1, Loss: 0.4636
Validation: {'Accuracy': 0.875, 'Precision': 0.875, 'Recall': 1.0, 'F1': 0.9333333333333333}
Epoch 2, Loss: 0.1936
Validation: {'Accuracy': 0.9955357142857143, 'Precision': 0.9949238578680203, 'Recall': 1.0, 'F1': 0.9974554707379135}
Epoch 3, Loss: 0.0834
Validation: {'Accuracy': 0.9910714285714286, 'Precision': 0.98989898989899, 'Recall': 1.0, 'F1': 0.9949238578680203}
Epoch 4, Loss: 0.0425
Validation: {'Accuracy': 0.9910714285714286, 'Precision': 0.98989898989899, 'Recall': 1.0, 'F1': 0.9949238578680203}
Epoch 5, Loss: 0.0251
Validation: {'Accuracy': 0.9598214285714286, 'Precision': 0.9947089947089947, 'Recall': 0.9591836734693877, 'F1': 0.9766233766233766}


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()  # no weight update while evaluating training metrics
train_preds = []
train_labels = []

with torch.no_grad():
    for img1, img2, label in train_loader:
        img1, img2, label = img1.to(device), img2.to(device), label.to(device)

        output = model(img1, img2).squeeze()
        preds = (torch.sigmoid(output) > 0.5).float()

        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(label.cpu().numpy())

# Compute training metrics
train_accuracy  = accuracy_score(train_labels, train_preds)
train_precision = precision_score(train_labels, train_preds)
train_recall    = recall_score(train_labels, train_preds)
train_f1        = f1_score(train_labels, train_preds)

print("✅ Training Metrics")
print(f"Accuracy:  {train_accuracy:.4f}")
print(f"Precision: {train_precision:.4f}")
print(f"Recall:    {train_recall:.4f}")
print(f"F1 Score:  {train_f1:.4f}")


✅ Training Metrics
Accuracy:  0.9719
Precision: 0.9855
Recall:    0.9824
F1 Score:  0.9839


In [10]:
import json

train_results = {
    "train_accuracy": round(train_accuracy, 4),
    "train_precision": round(train_precision, 4),
    "train_recall": round(train_recall, 4),
    "train_f1_score": round(train_f1, 4)
}

with open("results_task_b_train.json", "w") as f:
    json.dump(train_results, f, indent=4)

print("✅ Saved training metrics to results_task_b_train.json")


✅ Saved training metrics to results_task_b_train.json
